In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

from poliastro.bodies import Sun, Earth, Jupiter, Mars, Venus
from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import StaticOrbitPlotter
from poliastro.util import norm, time_range
from poliastro import iod
from poliastro.plotting import OrbitPlotter3D
from poliastro.threebody.flybys import compute_flyby

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
## Main dates (1 набор дат) 1762 дней полета
##date_launch = Time("2026-10-31", scale="utc").tdb
##date_flyby_Mars = Time("2027-08-08", scale="utc").tdb
##date_flyby_Venus = Time("2028-09-14", scale="utc").tdb
##date_arrival = Time("2031-08-28", scale="utc").tdb

## Main dates (2 набор дат) 1282 дня полета 
date_launch = Time("2028-11-17", scale="utc").tdb
date_flyby_Mars = Time("2029-08-17", scale="utc").tdb
date_flyby_Venus = Time("2030-09-24", scale="utc").tdb
date_arrival = Time("2032-05-22", scale="utc").tdb

In [3]:
earth_1 = Ephem.from_body(Earth, time_range(date_launch, end=date_flyby_Mars))
mar = Ephem.from_body(Mars, time_range(date_launch, end=date_flyby_Mars))
ven = Ephem.from_body(Venus, time_range(date_launch, end=date_flyby_Venus))
jup = Ephem.from_body(Jupiter, time_range(date_launch, end=date_arrival))

In [4]:
ss_earth_1 = Orbit.from_ephem(Sun, earth_1, date_launch)
ss_mar = Orbit.from_ephem(Sun, mar, date_flyby_Mars)
ss_ven = Orbit.from_ephem(Sun, ven, date_flyby_Venus)
ss_jup = Orbit.from_ephem(Sun, jup, date_arrival)

In [5]:
man_lambert_1 = Maneuver.lambert(ss_earth_1, ss_mar)
ss_trans_1, ss_target_1 = ss_earth_1.apply_maneuver(man_lambert_1, intermediate=True)

man_lambert_2 = Maneuver.lambert(ss_mar, ss_ven)
ss_trans_2, ss_target_2 = ss_mar.apply_maneuver(man_lambert_2, intermediate=True)

man_lambert_3 = Maneuver.lambert(ss_ven, ss_jup)
ss_trans_3, ss_target_3 = ss_ven.apply_maneuver(man_lambert_3, intermediate=True)

In [6]:
#2
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")


plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=186 * u.deg), color="blue", label="Transfer orbit E-M"
)

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=410 * u.deg), color="purple", label="Transfer orbit M-V"
)

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=164 * u.deg), color="crimson", label="Transfer orbit V-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [15]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")

plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=186 * u.deg), color="blue", label="Transfer orbit E-M"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [19]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(mar, date_flyby_Mars, label="Mars")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=410 * u.deg), color="purple", label="Transfer orbit M-V"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [21]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=164 * u.deg), color="crimson", label="Transfer orbit V-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [22]:
r_earth, v_earth = Ephem.from_body(Earth, date_launch).rv(date_launch)
r_mar, v_mar = Ephem.from_body(Mars, date_flyby_Mars).rv(date_flyby_Mars)
r_ven, v_ven = Ephem.from_body(Venus, date_flyby_Venus).rv(date_flyby_Venus)
r_jup, v_jup = Ephem.from_body(Jupiter, date_arrival).rv(date_arrival)

In [23]:
# Фазовые координаты Земли_1

print (f"{r_earth [0]}")
print (f"{r_earth [1]}")
print (f"{r_earth [2]}"'\n')

print (v_earth.to(u.km / u.s))

85521874.64962433 km
110624084.39718273 km
47957643.45103574 km

[-24.82670705  15.68194281   6.79697339] km / s


In [24]:
# Фазовые координаты Марса

print (f"{r_mar [0]}")
print (f"{r_mar [1]}")
print (f"{r_mar [2]}"'\n')

print (v_mar.to(u.km / u.s))

-56371033.88060116 km
-196382122.37251326 km
-88546371.47259791 km

[24.34257323 -3.45431803 -2.24108232] km / s


In [25]:
# Фазовые координаты Венеры

print (f"{r_ven [0]}")
print (f"{r_ven [1]}")
print (f"{r_ven [2]}"'\n')

print (v_ven.to(u.km / u.s))

-103269874.72373566 km
24938656.08209722 km
17757159.00885664 km

[ -9.88930581 -31.05491047 -13.34903441] km / s


In [26]:
# Фазовые координаты Юпитера

print (f"{r_jup [0]}")
print (f"{r_jup [1]}")
print (f"{r_jup [2]}"'\n')

print (v_jup.to(u.km / u.s))

286424322.12057704 km
-655753325.5249339 km
-288036968.94230103 km

[11.97429538  5.13218082  1.90826802] km / s


In [27]:
nh_tof_1 = date_flyby_Mars - date_launch # время полета Земля1 - Венера

(v0_Ea1, vk_mar), = iod.lambert(Sun.k, r_earth, r_mar, nh_tof_1)
nh_tof_1

<TimeDelta object: scale='tdb' format='jd' value=273.00000000104046>

In [28]:
# Гиперболический избыток скорости при отлете от Земли
V_inf = (norm(v0_Ea1 - v_earth)).to(u.km / u.s)
V_inf

<Quantity 3.18364521 km / s>

In [29]:
V_inf_vec = (v0_Ea1 - v_earth).to(u.km / u.s)
V_inf_vec

<Quantity [-2.86513408, -0.08420286,  1.38546504] km / s>

In [30]:
# Гиперболический избыток скорости при подлете к Марсу
V_inf_end = (norm(vk_mar - v_mar)).to(u.km / u.s)
V_inf_end

<Quantity 3.41766973 km / s>

In [31]:
V_inf_end_vec = (vk_mar - v_mar).to(u.km / u.s)
V_inf_end_vec

<Quantity [-3.22476815, -0.98365537, -0.56014185] km / s>

In [32]:
nh_tof_2 = date_flyby_Venus - date_flyby_Mars # время полета Венера - Земля2

(v0_mar, vk_ven), = iod.lambert(Sun.k, r_mar, r_ven, nh_tof_2)
nh_tof_2

<TimeDelta object: scale='tdb' format='jd' value=402.9999999939248>

In [33]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_end_0 = (norm(v0_mar - v_mar)).to(u.km / u.s)
V_inf_end_0

<Quantity 13.00643537 km / s>

In [34]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_end_0_vec = (v0_mar - v_mar).to(u.km / u.s)
V_inf_end_0_vec

<Quantity [-7.89711976, -9.22424917, -4.66005235] km / s>

In [35]:
# Гиперболический избыток скорости при подлете к Венере

V_inf_end_2 = (norm(vk_ven - v_ven)).to(u.km / u.s)
V_inf_end_2

<Quantity 11.75287775 km / s>

In [36]:
V_inf_end_2_vec = (vk_ven - v_ven).to(u.km / u.s)
V_inf_end_2_vec

<Quantity [-11.55851513,  -1.95972402,  -0.8308701 ] km / s>

In [37]:
nh_tof_3 = date_arrival - date_flyby_Venus # время полета Земля2 - Юпитер

(V0_ven, Vk_Jup), = iod.lambert(Sun.k, r_ven, r_jup, nh_tof_3)
nh_tof_3

<TimeDelta object: scale='tdb' format='jd' value=606.0000000321305>

In [38]:
# Гиперболический избыток скорости при отлете от Венеры

V_inf_end_2_0 = (norm(V0_ven - v_ven)).to(u.km / u.s)
V_inf_end_2_0

<Quantity 16.26358747 km / s>

In [39]:
# Гиперболический избыток скорости при отлете от Венеры

V_inf_end_2_vec_0 = (V0_ven - v_ven).to(u.km / u.s)
V_inf_end_2_vec_0

<Quantity [-14.98418803,  -6.3227502 ,  -0.03487489] km / s>

In [40]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_3_k = (norm(Vk_Jup - v_jup)).to(u.km / u.s)
V_inf_3_k

<Quantity 9.51351237 km / s>

In [41]:
V_inf_3_k_vec = (Vk_Jup - v_jup).to(u.km / u.s)
V_inf_3_k_vec

<Quantity [-3.9479447 , -7.8659162 , -3.61220328] km / s>

In [42]:
time_adv = nh_tof_1 + nh_tof_2 + nh_tof_3 
time_adv

<TimeDelta object: scale='tdb' format='jd' value=1282.0000000270957>

In [43]:
scalar_EM = r_earth[0]*r_mar[0] + r_earth[1]*r_mar[1] +r_earth[2]*r_mar[2] 
scalar_MV = r_ven[0]*r_mar[0] + r_ven[1]*r_mar[1] +r_ven[2]*r_mar[2]
scalar_VJ = r_ven[0]*r_jup[0] + r_ven[1]*r_jup[1] +r_ven[2]*r_jup[2]

In [44]:
fi_EM = (scalar_EM/norm(r_earth)/norm(r_mar))
fi_MV = (scalar_MV/norm(r_mar)/norm(r_ven))
fi_VJ = (scalar_VJ/norm(r_ven)/norm(r_jup))

In [45]:
np.degrees(np.arccos(fi_EM))

<Quantity 159.30261396 deg>

In [46]:
np.degrees(np.arccos(fi_MV))

<Quantity 91.54913118 deg>

In [47]:
np.degrees(np.arccos(fi_VJ))

<Quantity 127.90754821 deg>

In [48]:
## Необходимый импульс на опорной орбите 
from math import*

mu = 398600.4415 ## Гравитационный параметр Земли 
r = 6371.
R = r + 200.
V_gip = 3.1619866 ## Гиперболический избыток скорости при отлете от Земли

dV = sqrt (2 * mu / R + V_gip**2 ) - sqrt(mu/r)
dV * u.km / u.s

<Quantity 3.54966873 km / s>

In [49]:
## Необходимый импульс для торможения на эллиптическую орбиту Юпитера 
## Не уверен что это нужно

muJ = 126686534
r_p = 670900
r_a = 4000000
V_inf_jup = 6.7509575

print (sqrt ((2 * mu / R + V_inf_jup**2 )))
print (sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p))))

dV_el = sqrt ((2 * mu / R + V_inf_jup**2 )) - sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p)))
dV_el * u.km / u.s

12.918843495438685
17.983782441612995


<Quantity -5.06493895 km / s>

In [50]:
## параметры участка Землия-Марс 

from math import*

to_rad = pi/180

ist_an_1 = 343.3253220 * to_rad ##1) 356.4477161
ist_an_2 = 183.3673056 * to_rad ##1) 186.4941861

exs_EM = (norm(r_mar)/norm(r_earth) - 1) / (cos(ist_an_1) - (norm(r_mar)/norm(r_earth) * cos(ist_an_2)))
p_EM = norm(r_earth)*(1+cos(ist_an_1)*exs_EM)
a_EM = p_EM/(1 - exs_EM*exs_EM)

print (f"Эксцентриситет {exs_EM}")
print (f"Фокальный параметр {p_EM}")
print (f"Большая полуось {a_EM}")

Эксцентриситет 0.2056952401751984
Фокальный параметр 176950776.58192396 km
Большая полуось 184768426.9935489 km


In [51]:
## параметры участка Марс-Венера 

ist_an_1_MV = 138.0606683 * to_rad ##1)  136.5433673
ist_an_2_MV = 46.8001167 * to_rad  ##1) 327.4296561

exs_MV = (norm(r_ven)/norm(r_mar) - 1) / (cos(ist_an_1_MV) - (norm(r_ven)/norm(r_mar) * cos(ist_an_2_MV)))
p_MV = norm(r_mar)*(1+cos(ist_an_1_MV)*exs_MV)
a_MV = p_MV/(1 - exs_MV*exs_MV)

print (f"Эксцентриситет {exs_MV}")
print (f"Фокальный параметр {p_MV}")
print (f"Большая полуось {a_MV}")

Эксцентриситет 0.48026919711814425
Фокальный параметр 143124342.02740055 km
Большая полуось 186034865.3280164 km


In [52]:
## параметры участка Венера-Юпитер

ist_an_1_VJ = 36.8271421 * to_rad  ##1)  333.5703820
ist_an_2_VJ = 164.3358371 * to_rad ##1)  184.5281640

exs_VJ = (norm(r_jup)/norm(r_ven) - 1) / (cos(ist_an_1_VJ) - (norm(r_jup)/norm(r_ven) * cos(ist_an_2_VJ)))
p_VJ = norm(r_ven)*(1+cos(ist_an_1_VJ)*exs_VJ)
a_VJ = p_VJ/(1 - exs_VJ*exs_VJ)

print (f"Эксцентриситет {exs_VJ}")
print (f"Фокальный параметр {p_VJ}")
print (f"Большая полуось {a_VJ}")

Эксцентриситет 0.800611129957369
Фокальный параметр 176739277.61562008 km
Большая полуось 492280046.68499005 km


In [53]:
## Вектор гиперболического избытка скорости после грав. маневра у Марса

r_pi_Mar = 3735.9689
muMar = 42828

V_inf_Mar = 3.4176697
V_inf_Mar_vec_t_0 = -3.2247681
V_inf_Mar_vec_t_1 = -0.98365537
V_inf_Mar_vec_t_2 = -0.56014185

betta_Mar = 2*asin(1/(1+((r_pi_Mar*V_inf_Mar*V_inf_Mar)/muMar)))
gamma_Mar = pi
Vxy_inf_Mar = sqrt (V_inf_Mar_vec_t_0**2 + V_inf_Mar_vec_t_1**2)

V_grav_X_Mar = V_inf_Mar_vec_t_0 * cos(betta_Mar) - (V_inf_Mar_vec_t_1*V_inf_Mar)/Vxy_inf_Mar * sin(betta_Mar)*cos(gamma_Mar) - (V_inf_Mar_vec_t_0*V_inf_Mar_vec_t_2)/Vxy_inf_Mar * sin(betta_Mar)*sin(gamma_Mar)
V_grav_Y_Mar = V_inf_Mar_vec_t_1 * cos(betta_Mar) + (V_inf_Mar_vec_t_0*V_inf_Mar)/Vxy_inf_Mar * sin(betta_Mar)*cos(gamma_Mar) - (V_inf_Mar_vec_t_1*V_inf_Mar_vec_t_2)/Vxy_inf_Mar * sin(betta_Mar)*sin(gamma_Mar)
V_grav_Z_Mar = V_inf_Mar_vec_t_2 * cos(betta_Mar) + Vxy_inf_Mar * sin(betta_Mar)*sin(gamma_Mar)
V_grav_Mar = sqrt (V_grav_X_Mar**2 + V_grav_Y_Mar**2 + V_grav_Z_Mar**2) 

print (f"betta_Mar      {betta_Mar*180/pi} deg")
print (f"V_grav_X_Mar   {V_grav_X_Mar * u.km / u.s}")
print (f"V_grav_Y_Mar   {V_grav_Y_Mar * u.km / u.s}")
print (f"V_grav_Z_Mar   {V_grav_Z_Mar * u.km / u.s}")
print (f"V_grav_Mar     {V_grav_Mar * u.km / u.s}")

betta_Mar      59.38129708660475 deg
V_grav_X_Mar   -2.500560235212295 km / s
V_grav_Y_Mar   2.312200834439805 km / s
V_grav_Z_Mar   -0.2852927680933103 km / s
V_grav_Mar     3.4176696961870734 km / s


In [54]:
## Вектор гиперболического избытка скорости после грав. маневра у Венеры

r_pi_ven = 13084.2212
muVen = 324859

V_inf_Ven = 11.752878
V_inf_end_vec_t_0 = -11.558515
V_inf_end_vec_t_1 = -1.959724
V_inf_end_vec_t_2 = -0.8308701

betta_ven = 2*asin(1/(1+((r_pi_ven*V_inf_Ven*V_inf_Ven)/muVen)))
gamma_ven = pi
Vxy_inf_ven = sqrt (V_inf_end_vec_t_0**2 + V_inf_end_vec_t_1**2)

V_grav_X_Ven = V_inf_end_vec_t_0 * cos(betta_ven) - (V_inf_end_vec_t_1*V_inf_Ven)/Vxy_inf_ven * sin(betta_ven)*cos(gamma_ven) - (V_inf_end_vec_t_0*V_inf_end_vec_t_2)/Vxy_inf_ven * sin(betta_ven)*sin(gamma_ven)
V_grav_Y_Ven = V_inf_end_vec_t_1 * cos(betta_ven) + (V_inf_end_vec_t_0*V_inf_Ven)/Vxy_inf_ven * sin(betta_ven)*cos(gamma_ven) - (V_inf_end_vec_t_1*V_inf_end_vec_t_2)/Vxy_inf_ven * sin(betta_ven)*sin(gamma_ven)
V_grav_Z_Ven = V_inf_end_vec_t_2 * cos(betta_ven) + Vxy_inf_ven * sin(betta_ven)*sin(gamma_ven)
V_grav_Ven = sqrt (V_grav_X_Ven**2 + V_grav_Y_Ven**2 + V_grav_Z_Ven**2) 

print (f"betta_ven      {betta_ven*180/pi} deg")
print (f"V_grav_X_Ven   {V_grav_X_Ven * u.km / u.s}")
print (f"V_grav_Y_Ven   {V_grav_Y_Ven * u.km / u.s}")
print (f"V_grav_Z_Ven   {V_grav_Z_Ven * u.km / u.s}")
print (f"V_grav_Ven     {V_grav_Ven * u.km / u.s}")

betta_ven      17.527400918071734 deg
V_grav_X_Ven   -11.613562750805073 km / s
V_grav_Y_Ven   1.6209754647212167 km / s
V_grav_Z_Ven   -0.7922953215195125 km / s
V_grav_Ven     11.752877651903681 km / s
